In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime
import glob
import numpy as np

In [36]:
for page_number in range(9, 12):
    # Para cada página, adiciona os links dos discursos em `links`
    print(f'Obtendo links: página {page_number}', end='\r')
    site_data = requests.get(base_url.format(page_number=page_number))
    soup = BeautifulSoup(site_data.content, 'html.parser')
    link_tags = soup.find_all('a', href=re.compile('TextoHTML'))
    for tag in link_tags:
        links.append(re.sub(r"\s", "", tag['href']))

In [9]:
link_data = requests.get(base_link+links[1])
link_soup = BeautifulSoup(link_data.content, 'html.parser')
content = link_soup.find('div', id='content')

In [15]:
def process(df, content):    
    new_content = {'Discurso': content.find(align="justify").get_text(),
                  'Sessão': re.search(re.compile("Sessão: ([\w.]+)"),
                      content.find('td', text=re.compile('Sessão:')).get_text().strip()).group(1),
                  'Fase': re.search(re.compile("Fase: ([\w.]+)"),
                      content.find('td', text=re.compile('Fase:')).get_text().strip()).group(1),
                  'Data': re.search(re.compile("Data: ([\w/]+)"),
                      content.find('td', text=re.compile('Data:')).get_text().strip()).group(1)}
    
    df = df.append(new_content, ignore_index=True)
    return df

def vazio(df):
    null_content = {'Discurso': None,
                  'Sessão': None,
                  'Fase': None,
                  'Data': None}
    df = df.append(null_content, ignore_index=True)
    return df

def main():
    # Armazena discursos do site da câmara em um único dataframe.
    #base_url = 'https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?CurrentPage={page_number}&BasePesq=plenario&txIndexacao=&txOrador=&txPartido=&dtInicio=01/01/2017&dtFim=31/12/2017&txUF=&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=DESC&PageSize=50&txTexto=&txSumario='
    base_url = 'https://www.camara.leg.br/internet/sitaqweb/resultadoPesquisaDiscursos.asp?txIndexacao=&CurrentPage={page_number}&BasePesq=plenario&txOrador=&txPartido=&dtInicio=01/01/2007&dtFim=31/12/2007&txUF=&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=DESC&PageSize=50&txTexto=&txSumario='
    base_link = 'https://www.camara.leg.br/internet/sitaqweb/'
    links = list()
    oradores_tag = []
    oradores=[]
    partidos = []
    pattern = re.compile('[A-ZÀÁÂĖÈÉÊÌÍÒÓÔÕÙÚÛÇ]+(?:[ ]|-|,|(?:das?|dos?|de|e|\(|\)|[A-ZÀÁÂÃĖÈÉÊÌÍÒÓÔÕÙÚÛÇ]+))*')
    pattern_p = re.compile('[A-Z]+-(?:[A-Z]+)*')
    df = pd.DataFrame({'Orador': [], 'Partido': [],'Discurso': [], 'Sessão': [], 'Data': [], 'Fase': [], 'Link': []})

    # TODO obter automaticamente o número de p
    # áginas
    # ou ir avançando as páginas até não encontrar mais.
    
    for page_number in range(1, 504):
        # Para cada página, adiciona os links dos discursos em `links`
        print(f'Obtendo links: página {page_number}', end='\r')
        site_data = requests.get(base_url.format(page_number=page_number))
        soup = BeautifulSoup(site_data.content, 'html.parser')
        link_tags = soup.find_all('a', href=re.compile('TextoHTML'))
        for tag in link_tags:
            links.append(re.sub(r"\s", "", tag['href']))
        tabela = soup.find('table', class_='table table-bordered variasColunas')
        for row in tabela.findAll("tr"): #para tudo que estiver em <tr>
            cells = row.findAll('td') #variável para encontrar <td>
            if len(cells)==8: #número de colunas
                oradores_tag.append(cells[5].find(text=True)) #iterando sobre cada linha 
    for tag in oradores_tag:
        oradores.append(str(pattern.findall(tag)))
        partidos.append(str(pattern_p.findall(tag)))  
    print()
    
    # Salva os links caso dê algum problema no caminho.
    #with open('links_discursos_2013.txt', 'w') as f:
     #   f.write('\n'.join(links))
        
    n_links = len(links)
    links_erro = list()
    links_discursos = list()
    print(f'Encontrados {n_links} links.')
    print('Extraindo discursos...')
    for n, link in enumerate(links):
        link_data = requests.get(base_link+link)
        content = BeautifulSoup(link_data.content, 'html.parser')
        #content = link_soup.find('div', id='content')
        if content:
            try:
                df = process(df, content)
                links_discursos.append(base_link+link)
            except:
            # Salva os links com erro.
                df = vazio(df)
                links_erro.append(base_link+link)
                links_discursos.append(base_link+link)
                print(f'{n+1} discursos com erro.', end='\r')
        else:
            df = vazio(df)
            links_discursos.append(base_link+link)
        print(f'{n+1} discursos de {n_links} extraídos.', end='\r')
    
    with open('links_erro_2007.txt', 'w') as f:
        f.write('\n'.join(links_erro))
    print()
    df['Orador'] = oradores
    df['Partido'] = partidos
    df['Link'] = links_discursos

    return df

if __name__ == '__main__':
    df = main()
    df.to_csv('discursos\camara_2007_novo.csv', index=False, sep="|")

Obtendo links: página 503
Encontrados 25116 links.
Extraindo discursos...
25116 discursos de 25116 extraídos.


In [3]:
csv_files = glob.glob('discursos\camara_*_novo.csv')

# This creates a list of dataframes
df_list = (pd.read_csv(file, sep="|") for file in csv_files)

# Concatenate all DataFrames
discursos   = pd.concat(df_list, ignore_index=True)


In [2]:
discursos = pd.read_csv('discursos\discursos_completo.csv', sep="|")

In [12]:
def process(df, content):    
    new_content = {'Discurso': content.find(align="justify").get_text(),
                  'Sessão': re.search(re.compile("Sessão: ([\w.]+)"),
                      content.find('td', text=re.compile('Sessão:')).get_text().strip()).group(1),
                  'Fase': re.search(re.compile("Fase: ([\w.]+)"),
                      content.find('td', text=re.compile('Fase:')).get_text().strip()).group(1),
                  'Data': re.search(re.compile("Data: ([\w/]+)"),
                      content.find('td', text=re.compile('Data:')).get_text().strip()).group(1)}
    
    df = df.append(new_content, ignore_index=True)
    return df

def vazio(df):
    null_content = {'Discurso': None,
                  'Sessão': None,
                  'Fase': None,
                  'Data': None}
    df = df.append(null_content, ignore_index=True)
    return df

nulos = discursos[discursos['Data'].isna()]
pegar_discursos = nulos[nulos['Link'].notnull()]
links = pegar_discursos['Link'].values.tolist()
df = pd.DataFrame({'Orador': [], 'Partido': [],'Discurso': [], 'Sessão': [], 'Data': [], 'Fase': [], 'Link': []})

n_links = len(links)
links_erro = list()
links_discursos = list()
print(f'Encontrados {n_links} links.')
print('Extraindo discursos...')
for n, link in enumerate(links):
    link_data = requests.get(link)
    content = BeautifulSoup(link_data.content, 'html.parser')
    #content = link_soup.find('div', id='content')
    if content:
        try:
            df = process(df, content)
            links_discursos.append(link)
        except:
        # Salva os links com erro.
            df = vazio(df)
            links_erro.append(link)
            links_discursos.append(link)
            print(f'{n+1} discursos com erro.', end='\r')
    else:
        df = vazio(df)
        links_discursos.append(link)
    print(f'{n+1} discursos de {n_links} extraídos.', end='\r')

with open('links_erro_pegar_discursos.txt', 'w') as f:
    f.write('\n'.join(links_erro))
print()
df['Link'] = links_discursos

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336251 entries, 0 to 336250
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Orador    336251 non-null  object
 1   Partido   336251 non-null  object
 2   Discurso  334091 non-null  object
 3   Sessão    334107 non-null  object
 4   Data      334107 non-null  object
 5   Fase      334107 non-null  object
 6   Link      272432 non-null  object
dtypes: object(7)
memory usage: 18.0+ MB


In [26]:
link_data = requests.get("https://www.camara.leg.br/internet/sitaqweb/TextoHTML.asp?etapa=5&nuSessao=94.2019&nuQuarto=115863&nuOrador=1&nuInsercao=1&dtHorarioQuarto=20:12&sgFaseSessao=EN&Data=08/05/2019&txApelido=RONALDOCARLETTO,PP-BA&txFaseSessao=Encerramento&txTipoSessao=DeliberativaOrdinária-CD&dtHoraQuarto=20:12&txEtapa=")
content = BeautifulSoup(link_data.content, 'html.parser')
content.find(align="justify").get_text()
re.search(re.compile("Sessão: ([\w.]+)"),content.find('td', text=re.compile('Sessão:')).get_text().strip()).group(1)
re.search(re.compile("Fase: ([\w.]+)"),content.find('td', text=re.compile('Fase:')).get_text().strip()).group(1)
re.search(re.compile("Data: ([\w/]+)"),content.find('td', text=re.compile('Data:')).get_text().strip()).group(1)

if content:
    df = process(df, content)
    links_discursos.append(base_link+link)

In [35]:
pegar = pd.merge(pegar_discursos,df, on = "Link", how="inner")

In [51]:
pegar_2 = pegar_discursos.join(df.set_index("Link"), on = "Link", how="inner",lsuffix="_x",rsuffix="_y")
pegar_2 = pegar_2[['Discurso_y', 'Sessão_y', 'Data_y', 'Fase_y','Link']]
pegar_2 = pegar_2.rename(columns={'Discurso_y':'Discurso','Sessão_y':'Sessão', 'Data_y':'Data', 'Fase_y':'Fase'})

discursos_2 = discursos.join(pegar_2.set_index("Link"), on = "Link", how="left",lsuffix="_x",rsuffix="_y")

In [47]:
pegar_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1707 entries, 90382 to 336250
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Discurso  1177 non-null   object
 1   Sessão    1177 non-null   object
 2   Data      1177 non-null   object
 3   Fase      1177 non-null   object
 4   Link      1707 non-null   object
dtypes: object(5)
memory usage: 80.0+ KB


In [48]:
discursos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336251 entries, 0 to 336250
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   Orador    336251 non-null  object
 1   Partido   336251 non-null  object
 2   Discurso  334091 non-null  object
 3   Sessão    334107 non-null  object
 4   Data      334107 non-null  object
 5   Fase      334107 non-null  object
 6   Link      272432 non-null  object
dtypes: object(7)
memory usage: 18.0+ MB


In [33]:
df.to_csv('df.csv',sep="}")
pegar_discursos.to_csv("pegar_discursos.csv",sep="}")

In [14]:
df = pd.read_csv('df.csv', sep="}")
pegar_discursos = pd.read_csv('pegar_discursos.csv', sep="}")

In [15]:
df = df[['Discurso', 'Sessão', 'Data', 'Fase','Link']]
df['Link'] = df['Link'].str[44:]
discursos_2 = discursos.join(df.set_index("Link"), on = "Link", how="left",lsuffix="_x",rsuffix="_y")

In [22]:
discursos_2[discursos_2['Discurso_y'].notnull()].head()

,Orador,Partido,Discurso_x,Sessão_x,Data_x,Fase_x,Link,Discurso_y,Sessão_y,Data_y,Fase_y
90484,"['DUARTE NOGUEIRA, PSDB-SP']",['PSDB-SP'],O SR. DUARTE NOGUEIRA (PSDB-SP. Pela ordem. Se...,273.1.54.O,05/10/2011,OD,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. DUARTE NOGUEIRA (PSDB-SP. Pela ordem. Se...,273.1.54.O,05/10/2011,OD
90498,"['ANTONIO CARLOS MAGALHÃES NETO, DEM-BA']",['DEM-BA'],O SR. ANTONIO CARLOS MAGALHÃES NE...,273.1.54.O,05/10/2011,OD,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. ANTONIO CARLOS MAGALHÃES NE...,273.1.54.O,05/10/2011,OD
90515,"['PAUDERNEY AVELINO, DEM-AM']",['DEM-AM'],O SR. PAUDERNEY AVELINO (DEM-AM. Pronuncia o s...,273.1.54.O,05/10/2011,PE,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. PAUDERNEY AVELINO (DEM-AM. Pronuncia o s...,273.1.54.O,05/10/2011,PE
90798,"['MANUELA D', 'ÁVILA, PCDOB-RS']",['PCDOB-RS'],A SRA. MANUELA D'ÁVILA (Bloco/PCdoB-RS. Pronun...,269.1.54.O,04/10/2011,PE,https://www.camara.leg.br/internet/sitaqweb/Te...,A SRA. MANUELA D'ÁVILA (Bloco/PCdoB-RS. Pronun...,269.1.54.O,04/10/2011,PE
90895,"['EMILIANO JOSÉ, PT-BA']",['PT-BA'],O SR. EMILIANO JOSÉ (PT-BA. Sem revisão do or...,267.1.54.O,03/10/2011,CP,https://www.camara.leg.br/internet/sitaqweb/Te...,O SR. EMILIANO JOSÉ (PT-BA. Sem revisão do or...,267.1.54.O,03/10/2011,CP


In [21]:
discursos_2['Discurso_x'] = np.where(discursos_2['Discurso_y'].notnull(),discursos_2['Discurso_y'],discursos_2['Discurso_x'])
discursos_2['Data_x'] = np.where(discursos_2['Data_y'].notnull(),discursos_2['Data_y'],discursos_2['Data_x'])
discursos_2['Sessão_x'] = np.where(discursos_2['Sessão_y'].notnull(),discursos_2['Sessão_y'],discursos_2['Sessão_x'])
discursos_2['Fase_x'] = np.where(discursos_2['Fase_y'].notnull(),discursos_2['Fase_y'],discursos_2['Fase_x'])

In [23]:
discursos_2.columns

Index(['Orador', 'Partido', 'Discurso_x', 'Sessão_x', 'Data_x', 'Fase_x',
       'Link', 'Discurso_y', 'Sessão_y', 'Data_y', 'Fase_y'],
      dtype='object')

In [24]:
discursos_2 = discursos_2[['Orador', 'Partido', 'Discurso_x', 'Sessão_x', 'Data_x', 'Fase_x','Link']]
discursos_2 = discursos_2.rename(columns={'Discurso_x':'Discurso', 'Sessão_x':'Sessão', 'Data_x':'Data', 'Fase_x':'Fase'})
discursos_2.head()

,Orador,Partido,Discurso,Sessão,Data_x,Fase,Link
0,"['MAURO BENEVIDES, PMDB-CE']",['PMDB-CE'],O SR. MAURO BENEVIDES (Bloco/PMDB-CE. Pronunci...,359.1.53.O,21/12/2007,CP,https://www.camara.leg.br/internet/sitaqweb/Te...
1,"['MAURO BENEVIDES (PRESIDENTE), PMDB-CE']",['PMDB-CE'],O SR. PRESIDENTE (Mauro Benevides) -...,359.1.53.O,21/12/2007,GE,https://www.camara.leg.br/internet/sitaqweb/Te...
2,"['EDUARDO GOMES, PSDB-TO']",['PSDB-TO'],O SR. EDUARDO GOMES (PSDB-TO. Como Líder. Sem...,359.1.53.O,21/12/2007,GE,https://www.camara.leg.br/internet/sitaqweb/Te...
3,"['MAURO BENEVIDES (PRESIDENTE), PMDB-CE']",['PMDB-CE'],O SR. PRESIDENTE (Mauro Benevides) ...,359.1.53.O,21/12/2007,GE,https://www.camara.leg.br/internet/sitaqweb/Te...
4,"['DARCÍSIO PERONDI, PMDB-RS']",['PMDB-RS'],O SR. DARCÍSIO PERONDI (Bloco/PMDB-RS. Como Lí...,359.1.53.O,21/12/2007,GE,https://www.camara.leg.br/internet/sitaqweb/Te...


In [26]:
discursos_2.to_csv('discursos\discursos_completo.csv', index=False, sep="|")